In [1]:
from google.colab import drive

# 구글 드라이브를 마운트
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch.nn as nn
import torch
from tqdm import tqdm
from torchvision.utils import save_image

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LR = 0.001
BATCH_SIZE = 128
IMAGE_SIZE = 64
IMG_CHANNELS = 3
LATENT_DIM = 100
NUM_EPOCHS = 200

In [4]:
class Generator(nn.Module):
  def __init__(self,img_size=64, latent_dim=100):
    super(Generator, self).__init__()

    self.init_size = img_size // 16
    # self.l1 = nn.Linear(latent_dim, 1024 * self.init_size ** 2)
    self.l1 = nn.ConvTranspose2d(in_channels=latent_dim, out_channels=1024, kernel_size=self.init_size, stride=1, padding=0)

    self.blocks = nn.Sequential(
     self.block(1024, 512, 5, 2, 2, 1),
     self.block(512, 256, 5, 2, 2, 1),
     self.block(256, 128, 5, 2, 2, 1),
     nn.ConvTranspose2d(128, 3, 5, 2, 2, 1),
     nn.Tanh(),
    )

    self.initialize_weight()

  def initialize_weight(self):
    for m in self.modules():
      if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)

  def block(self, in_channels, out_channels, kernel_size, stride, padding, output_padding, bias=False):
    result = nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ConvTranspose2d(in_channels,
                           out_channels,
                           kernel_size,
                           stride,
                           padding,
                           output_padding,
                           bias=bias,
        ),
        nn.ReLU(),
    )
    return result

  def forward(self, z):
    output = self.l1(z)
    # output = output.view(-1, 1024, 4, 4)
    output = self.blocks(output)
    return output

In [5]:
class Discriminator(nn.Module):
  def __init__(self, IMAGE_SIZE):
    super(Discriminator, self).__init__()

    self.disc_size = IMAGE_SIZE // 8

    self.disc_blocks = nn.Sequential(
        nn.Conv2d(3, 128, 5, 2, 2),
        nn.LeakyReLU(0.2),
        self.disc_block(128, 256, 5, 2, 2),
        self.disc_block(256, 512, 5, 2, 2),
        nn.Conv2d(512, 1, self.disc_size),
        nn.Flatten(),
        nn.Sigmoid(),
    )

    self.initialize_weight()

  def initialize_weight(self):
    for m in self.modules():
      if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)

  def disc_block(self, in_channels, out_channels, kernel_size, stride, padding, bias=False):
    result = nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=bias
        ),
        nn.LeakyReLU(0.2),
    )
    return result

  def forward(self, x):
    return self.disc_blocks(x)


### 데이터 세팅 및 학습준비

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jhoward/lsun_bedroom")

print("Path to dataset files:", path)

100%|██████████| 8.89G/8.89G [06:55<00:00, 23.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jhoward/lsun_bedroom/versions/1


In [7]:
!mv /root/.cache/kagglehub/datasets/ /content

mv: cannot move '/root/.cache/kagglehub/datasets/' to '/content/datasets': Directory not empty


In [8]:
import torchvision.datasets as vdatasets
import torchvision.transforms as T

In [9]:
dataset = vdatasets.ImageFolder(root='/content/datasets/jhoward/lsun_bedroom/versions/1/data0/lsun/bedroom',
                                transform=T.Compose([
                                    T.Resize(IMAGE_SIZE),
                                    T.CenterCrop(IMAGE_SIZE),
                                    T.ToTensor(),
                                    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=4)

one_batch = next(iter(dataloader))

In [10]:
for image,label in dataloader:
  break

In [11]:
# for i in range(10):
#   print(i)
#   for image,label in tqdm(dataloader):
#     pass

In [12]:
image.shape

torch.Size([128, 3, 64, 64])

In [13]:
gen = Generator(IMAGE_SIZE, LATENT_DIM).to(device)
disc = Discriminator(IMAGE_SIZE).to(device)

In [14]:
gen_optim = torch.optim.Adam(gen.parameters(), lr=LR, betas=(0.5, 0.999))
# momentum이라서
disc_optim = torch.optim.Adam(disc.parameters(), lr=LR, betas=(0.5, 0.999))
criterion = nn.BCELoss()

In [15]:
# train모드
gen.train()
disc.train()

Discriminator(
  (disc_blocks): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Conv2d(512, 1, kernel_size=(8, 8), stride=(1, 1))
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): Sigmoid()
  )
)

## 학습

In [16]:
save_interval = 100
sample_interval = len(dataloader)
save_path_gen_template = '/content/drive/MyDrive/generator_epoch_%d.pth'
save_path_disc_template = '/content/drive/MyDrive/discriminator_epoch_%d.pth'

for epoch in tqdm(range(NUM_EPOCHS)):
  for idx, (real_img, label) in enumerate(dataloader):
    real_img = real_img.to(device)
    latent_z_batch = torch.rand(BATCH_SIZE, LATENT_DIM, 1, 1).to(device)

    fake_img = gen(latent_z_batch)

    real_disc_pred = disc(real_img)
    real_disc_loss = criterion(real_disc_pred, torch.ones_like(real_disc_pred))

    fake_img_detach = fake_img.detach()
    fake_disc_pred = disc(fake_img_detach)
    fake_disc_loss = criterion(fake_disc_pred, torch.zeros_like(fake_disc_pred))

    disc_loss = (real_disc_loss + fake_disc_loss) / 2

    disc.zero_grad()
    disc_loss.backward() # gradient 구한다
    disc_optim.step() # weight update

    #---------------------------------
    fake_disc_pred_for_gen = disc(fake_img)
    gen_loss = criterion(fake_disc_pred_for_gen, torch.ones_like(fake_disc_pred_for_gen))
    gen.zero_grad()
    gen_loss.backward()
    gen_optim.step()

    #---------------------------------
    if idx%100 == 0:
      print(f"Batch {idx}/{len(dataloader)}, Disc_loss: {disc_loss.item()}, Gen_loss: {gen_loss.item()}")

    #---------------------------------

    # gan의 inference : random latent z에서 실제같은 이미지 만드는 것
    batches_done = epoch * len(dataloader) + idx
    if batches_done % sample_interval == 0:
      save_image(fake_img.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

  # 특정 에폭마다 가중치 저장
  if (epoch + 1) % save_interval == 0:
    gen_save_path = save_path_gen_template % (epoch + 1)
    disc_save_path = save_path_disc_template % (epoch + 1)
    torch.save(gen.state_dict(), gen_save_path)
    torch.save(disc.state_dict(), disc_save_path)
    print(f"Epoch {epoch+1} : model saved")


  0%|          | 0/200 [00:00<?, ?it/s]

Batch 0/2369, Disc_loss: 0.6926150321960449, Gen_loss: 2.9475343227386475
Batch 100/2369, Disc_loss: 0.36481988430023193, Gen_loss: 5.417544364929199
Batch 200/2369, Disc_loss: 1.2709417343139648, Gen_loss: 8.56411361694336
Batch 300/2369, Disc_loss: 1.102737545967102, Gen_loss: 22.280757904052734
Batch 400/2369, Disc_loss: 0.21278956532478333, Gen_loss: 4.299188613891602
Batch 500/2369, Disc_loss: 0.09044096618890762, Gen_loss: 3.4406991004943848
Batch 600/2369, Disc_loss: 0.16382579505443573, Gen_loss: 4.851631164550781
Batch 700/2369, Disc_loss: 0.22313328087329865, Gen_loss: 7.990861892700195
Batch 800/2369, Disc_loss: 0.09069117158651352, Gen_loss: 8.027162551879883
Batch 900/2369, Disc_loss: 0.223793163895607, Gen_loss: 15.86021900177002
Batch 1000/2369, Disc_loss: 0.08213486522436142, Gen_loss: 3.77934193611145
Batch 1100/2369, Disc_loss: 0.2806395888328552, Gen_loss: 15.308599472045898
Batch 1200/2369, Disc_loss: 0.056648723781108856, Gen_loss: 7.402921676635742
Batch 1300/2369

  0%|          | 1/200 [02:14<7:26:19, 134.57s/it]

Batch 0/2369, Disc_loss: 0.427120566368103, Gen_loss: 4.834680080413818
Batch 100/2369, Disc_loss: 0.618289589881897, Gen_loss: 7.997520446777344
Batch 200/2369, Disc_loss: 0.234977126121521, Gen_loss: 6.063907623291016
Batch 300/2369, Disc_loss: 0.24190625548362732, Gen_loss: 6.427026748657227
Batch 400/2369, Disc_loss: 0.22539293766021729, Gen_loss: 7.543214797973633
Batch 500/2369, Disc_loss: 0.1566651463508606, Gen_loss: 6.977808475494385
Batch 600/2369, Disc_loss: 0.34192246198654175, Gen_loss: 4.925686359405518
Batch 700/2369, Disc_loss: 0.268181174993515, Gen_loss: 10.045282363891602
Batch 800/2369, Disc_loss: 0.27607959508895874, Gen_loss: 6.28167724609375
Batch 900/2369, Disc_loss: 0.10097503662109375, Gen_loss: 6.470739364624023
Batch 1000/2369, Disc_loss: 0.4035758376121521, Gen_loss: 7.830252647399902
Batch 1100/2369, Disc_loss: 0.22053563594818115, Gen_loss: 4.454885482788086
Batch 1200/2369, Disc_loss: 0.22141575813293457, Gen_loss: 6.240447521209717
Batch 1300/2369, Disc

  1%|          | 2/200 [04:27<7:20:09, 133.38s/it]

...

Batch 0/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 100/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 200/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 300/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 400/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 500/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 600/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 700/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 800/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 900/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1000/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1100/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1200/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1300/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1400/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1500/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1600/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1700/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1800/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1900/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 2000/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batc

100%|█████████▉| 199/200 [7:20:58<02:13, 133.31s/it]

Batch 0/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 100/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 200/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 300/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 400/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 500/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 600/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 700/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 800/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 900/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1000/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1100/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1200/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1300/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1400/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1500/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1600/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1700/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1800/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 1900/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batch 2000/2369, Disc_loss: 50.0, Gen_loss: 0.0
Batc

100%|██████████| 200/200 [7:23:11<00:00, 132.96s/it]

Epoch 200 : model saved
